In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Predição de Satisfação de Passageiro com a viagem aérea

Essa é uma tentativa de predição de satisfação de um passageiro com a viagem feita por avião. Os dados estão disponíveis no [Kaggle](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction). O *dataset* é composto por 24 *features* para prever o *target* que é o nível de satisfação de um passageiro entre neu.

Escolhi um dataset tabular pois queria algo que me permitisse testar vários tipos de modelos, coisa que acredito que não seria possível com um dataset de imagens pois, para mim, a principal abordagem seria "simplesmente" testar configurações de redes neurais convolucionais. Com dados tabulares, posso validar vários modelos, inclusive uma rede neural.

Assim sendo, é um problema que pode ser resolvido com modelos supervisionados, já que temos a informação do nível de satisfação de um passageiro.